# 6.4.2 用XGBoost实现

## 1.业务理解

In [1]:
# 略，详见教材内容。本文件为教材《数据分析理论与实践：基于经典算法及Python编程实现》（朝乐门主编，机械工业出版社，2022年）的配套代码。

## 2.数据读入

In [2]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn import metrics
from sklearn.model_selection import GridSearchCV

In [3]:
#数据导入 内置数据集（波士顿房价数据）
from sklearn.datasets import load_boston
boston = load_boston()

C:\Users\chaol\AppData\Roaming\Python\Python38\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np

        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California 

In [4]:
import sklearn
sklearn.__version__

'1.1.3'

In [5]:
#显示数据集字典的键
print(boston.keys())

dict_keys(['data', 'target', 'feature_names', 'DESCR', 'filename', 'data_module'])


In [6]:
#数据集的完整描述
print(boston.DESCR)

.. _boston_dataset:

Boston house prices dataset
---------------------------

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pu

In [7]:
#数据集的特征名称
print(boston.feature_names)

['CRIM' 'ZN' 'INDUS' 'CHAS' 'NOX' 'RM' 'AGE' 'DIS' 'RAD' 'TAX' 'PTRATIO'
 'B' 'LSTAT']


In [8]:
#数据形状
print(boston.data.shape)

(506, 13)


In [9]:
#将数据（data）与回归目标（target）转化为数据框类型
X= pd.DataFrame(boston.data, columns=boston.feature_names)
y= pd.DataFrame(boston.target, columns=['MEDV'])

In [10]:
#合并数据框
df= pd.concat([X, y], axis=1)
df

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67,22.4
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48,22.0


In [11]:
help(load_boston)

Help on function load_boston in module sklearn.datasets._base:

load_boston(*, return_X_y=False)
    DEPRECATED: `load_boston` is deprecated in 1.0 and will be removed in 1.2.
    
    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.
    
    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.
    
    In this special case, you can fetch the dataset from the original
    source::
    
        import pandas as pd
        import numpy as np
    
        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]
    
    Alternative datasets include the California housing 

## 3.数据理解

In [12]:
#查看数据基本信息
df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     506 non-null    float64
 1   ZN       506 non-null    float64
 2   INDUS    506 non-null    float64
 3   CHAS     506 non-null    float64
 4   NOX      506 non-null    float64
 5   RM       506 non-null    float64
 6   AGE      506 non-null    float64
 7   DIS      506 non-null    float64
 8   RAD      506 non-null    float64
 9   TAX      506 non-null    float64
 10  PTRATIO  506 non-null    float64
 11  B        506 non-null    float64
 12  LSTAT    506 non-null    float64
 13  MEDV     506 non-null    float64
dtypes: float64(14)
memory usage: 55.5 KB


In [13]:
#查看描述性统计信息
df.describe()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.613524,11.363636,11.136779,0.069170,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,356.674032,12.653063,22.532806
std,8.601545,23.322453,6.860353,0.253994,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946,91.294864,7.141062,9.197104
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,0.320000,1.730000,5.000000
25%,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,375.377500,6.950000,17.025000
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,391.440000,11.360000,21.200000
75%,3.677083,12.500000,18.100000,0.000000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000,396.225000,16.955000,25.000000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,37.970000,50.000000


## 4.数据准备

In [14]:
#划分训练集与测试集
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y
                                                    ,test_size=0.3
                                                    ,random_state=42)  
X_test

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
173,0.09178,0.0,4.05,0.0,0.510,6.416,84.1,2.6463,5.0,296.0,16.6,395.50,9.04
274,0.05644,40.0,6.41,1.0,0.447,6.758,32.9,4.0776,4.0,254.0,17.6,396.90,3.53
491,0.10574,0.0,27.74,0.0,0.609,5.983,98.8,1.8681,4.0,711.0,20.1,390.11,18.07
72,0.09164,0.0,10.81,0.0,0.413,6.065,7.8,5.2873,4.0,305.0,19.2,390.91,5.52
452,5.09017,0.0,18.10,0.0,0.713,6.297,91.8,2.3682,24.0,666.0,20.2,385.09,17.27
...,...,...,...,...,...,...,...,...,...,...,...,...,...
441,9.72418,0.0,18.10,0.0,0.740,6.406,97.2,2.0651,24.0,666.0,20.2,385.96,19.52
23,0.98843,0.0,8.14,0.0,0.538,5.813,100.0,4.0952,4.0,307.0,21.0,394.54,19.88
225,0.52693,0.0,6.20,0.0,0.504,8.725,83.0,2.8944,8.0,307.0,17.4,382.00,4.63
433,5.58107,0.0,18.10,0.0,0.713,6.436,87.9,2.3158,24.0,666.0,20.2,100.19,16.22


## 5.模型训练

In [15]:
#模型训练
model=XGBRegressor()
model.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [16]:
help(XGBRegressor)

Help on class XGBRegressor in module xgboost.sklearn:

class XGBRegressor(XGBModel, sklearn.base.RegressorMixin)
 |  XGBRegressor(*, objective: Union[str, Callable[[numpy.ndarray, numpy.ndarray], Tuple[numpy.ndarray, numpy.ndarray]], NoneType] = 'reg:squarederror', **kwargs: Any) -> None
 |  
 |  Implementation of the scikit-learn API for XGBoost regression.
 |  
 |  
 |  Parameters
 |  ----------
 |  
 |      n_estimators : int
 |          Number of gradient boosted trees.  Equivalent to number of boosting
 |          rounds.
 |  
 |      max_depth :  Optional[int]
 |          Maximum tree depth for base learners.
 |      learning_rate : Optional[float]
 |          Boosting learning rate (xgb's "eta")
 |      verbosity : Optional[int]
 |          The degree of verbosity. Valid values are 0 (silent) - 3 (debug).
 |      objective : typing.Union[str, typing.Callable[[numpy.ndarray, numpy.ndarray], typing.Tuple[numpy.ndarray, numpy.ndarray]], NoneType]
 |          Specify the learning ta

In [17]:
#默认参数模型预测结果y_pred
y_pred=model.predict(X_test)
y_pred

array([24.778605 , 33.818237 , 16.406374 , 21.919922 , 16.906565 ,
       22.777987 , 17.356476 , 13.187292 , 20.986588 , 21.142221 ,
       20.54319  , 17.121996 ,  9.536354 , 21.091461 , 19.060402 ,
       25.708874 , 20.233723 , 10.601592 , 47.86842  , 15.67546  ,
       24.634136 , 24.914402 , 13.297937 , 23.103683 , 17.29109  ,
       14.3942995, 21.942911 , 12.735511 , 19.092672 , 20.922606 ,
       20.738443 , 23.78592  , 23.2116   , 21.518799 , 13.740485 ,
       16.88793  , 31.67158  , 18.544281 , 22.393078 , 24.590498 ,
       14.612399 , 27.65536  , 48.32444  , 18.866236 , 22.937452 ,
       13.490369 , 15.246323 , 24.50742  , 19.210526 , 28.331783 ,
       22.187742 , 34.691166 , 16.465992 , 27.75426  , 48.369026 ,
       21.04288  , 16.283428 , 32.370747 , 22.358805 , 18.022444 ,
       22.479692 , 34.404568 , 30.451216 , 17.837831 , 24.42846  ,
       15.181897 , 13.548869 , 22.872335 , 29.11238  , 14.936779 ,
       20.495413 , 29.009253 , 10.46218  , 19.606794 , 21.0167

## 6.模型评价

In [18]:
#回归指标——平均绝对误差（MAE）、均方根误差（RMSE）、R_squared（决定系数）
print("MAE:",metrics.mean_absolute_error(y_test, y_pred))
print("RMSE:",metrics.mean_squared_error(y_test, y_pred))
    #True 返回 MSE 值，如果 False 返回 RMSE 值
print("R_squared:",metrics.r2_score(y_test, y_pred))

MAE: 2.091345859828748
RMSE: 9.238913131574064
R_squared: 0.876009538173606


## 7.模型调参

In [19]:
#以n_estimators、max_depth、subsample、colsample_bytree、learning_rate参数和值设置字典列表param_grid。设置cv参数值为5。
param_grid={'n_estimators':[50,150,300]
            ,'max_depth':[3,4,5,6]
            ,'subsample':[0.8,0.9,1]
            ,'colsample_bytree':[0.8,0.9,1]
            ,'learning_rate':[0.05,0.1,0.2,0.3]}

In [20]:
#调用GridSearchCV函数，进行5折交叉验证，对估计器XGBRegressor()的指定参数值param_grid进行详尽搜索，得到最终的最优模型参数。
n_folds=5
estimator=GridSearchCV(model
                       ,param_grid
                       ,cv=n_folds)
estimator.fit(X_train,y_train)

GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                    colsample_bylevel=1, colsample_bynode=1,
                                    colsample_bytree=1,
                                    enable_categorical=False, gamma=0,
                                    gpu_id=-1, importance_type=None,
                                    interaction_constraints='',
                                    learning_rate=0.300000012, max_delta_step=0,
                                    max_depth=6, min_child_weight=1,
                                    missing=nan, monotone_constraints='()',
                                    n_estimators=100, n_jobs=8,
                                    num_parallel_tree=1, predictor='auto',
                                    random_state=0, reg_alpha=0, reg_lambda=1,
                                    scale_pos_weight=1, subsample=1,
                                    tree_method='exact', validate_parameters=1,
                                    verbosity=None),
             param_grid={'colsample_bytree': [0.8, 0.9, 1],
                         'learning_rate': [0.05, 0.1, 0.2, 0.3],
                         'max_depth': [3, 4, 5, 6],
                         'n_estimators': [50, 150, 300],
                         'subsample': [0.8, 0.9, 1]})

In [21]:
#通过best_estimator_属性，得出通过搜索选择的最高分（或最小损失）的估计量。
estimator.best_estimator_

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.1, max_delta_step=0,
             max_depth=4, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=150, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=0.9,
             tree_method='exact', validate_parameters=1, verbosity=None)

## 8.模型预测

In [22]:
#调参后的模型训练
model1=XGBRegressor(n_estimators=150
                    ,max_depth=4
                    ,subsample=0.8
                    ,colsample_bytree=0.9
                    ,learning_rate=0.2)

model1.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.9, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.2, max_delta_step=0,
             max_depth=4, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=150, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=0.8,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [23]:
#调参后的模型预测结果y_pred1
y_pred1=model1.predict(X_test)
y_pred1

array([23.971243 , 31.425901 , 15.314428 , 22.585379 , 17.813051 ,
       21.250525 , 19.038698 , 15.157877 , 21.50425  , 20.579308 ,
       22.247862 , 20.379448 ,  7.502841 , 20.492863 , 19.31373  ,
       25.757782 , 19.48982  , 11.632175 , 47.04139  , 14.201586 ,
       23.66073  , 24.684294 , 13.652991 , 22.00112  , 16.911795 ,
       15.945735 , 21.332262 , 12.430806 , 21.17626  , 21.612076 ,
       20.133194 , 23.498665 , 25.346174 , 19.95246  , 15.76574  ,
       16.13029  , 32.220676 , 18.30947  , 21.83184  , 22.833538 ,
       17.539953 , 29.415789 , 47.089924 , 20.856749 , 22.579573 ,
       15.281133 , 15.37613  , 24.048018 , 20.157608 , 29.100344 ,
       22.274933 , 36.416576 , 17.688187 , 26.428211 , 49.09732  ,
       21.251198 , 15.710475 , 33.000767 , 21.50849  , 18.830923 ,
       25.295229 , 34.676544 , 31.694492 , 19.685415 , 23.187723 ,
       18.338158 , 12.336575 , 23.989517 , 27.995363 , 16.405788 ,
       20.492998 , 25.099787 , 11.441424 , 19.627535 , 22.1469

In [24]:
#调参后的模型回归指标评价结果
print("MAE1:",metrics.mean_absolute_error(y_test, y_pred1))
print("RMSE1:",metrics.mean_squared_error(y_test, y_pred1,squared = False))
print("R_squared1:",metrics.r2_score(y_test, y_pred1))

MAE1: 1.9471647231202378
RMSE1: 2.9655804511684813
R_squared1: 0.8819715199680248
